In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np 

import osmnx as ox
from meteostat import Point, Daily, Hourly, Stations
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt

import plotly.express as px

import sys
sys.path.append("../src/data")
sys.path.append("../src/visualization")
import get_data as data_tools
import plots as plotting_tools

In [2]:
heatwave_stats = pd.read_csv("../data/interim/heatwave_stats_de_10year.csv")

In [6]:
heatwave_stats.loc[:, ["country", "location"]]

,country,location
0,DE,Berlin
1,DE,Hamburg
2,DE,Muenchen
3,DE,Koeln
4,DE,Frankfurt am Main
...,...,...
75,DE,Moers
76,DE,Salzgitter
77,DE,Siegen
78,DE,Guetersloh


In [4]:
# Initialize an empty GeoDataFrame to store the border geometries
city_boundaries = ox.geocode_to_gdf("Berlin, Germany").loc[:, ["name", "geometry"]]  # Just to create an empty GeoDataFrame with the correct schema

def get_geometry(name):
    try:
        gdf = ox.geocode_to_gdf(name).to_crs("epsg:3857")
        return gdf["geometry"]
    except:
        return None
    
# Iterate over each location, geocode it, and retrieve the border geometry
for index, row in heatwave_stats.iterrows():
    query = f"{row['location']}, {row['country']}"
    geometry = get_geometry(query)
    city_boundaries = pd.concat([city_boundaries, gpd.GeoDataFrame({'name': row['location'], 'geometry': geometry}, index=[0])])

city_boundaries

ValueError: Cannot determine common CRS for concatenation inputs, got ['WGS 84', 'WGS 84 / Pseudo-Mercator']. Use `to_crs()` to transform geometries to the same CRS before merging.

In [ ]:
# Plotting choropleth map
sub_stats = heatwave_stats.iloc[:5, :]
sub_geo = city_boundaries.iloc[:5, :]

print(sub_stats)
print(sub_geo)
fig = px.choropleth(sub_stats, 
                            geojson=sub_geo, 
                            locations=sub_stats.location,
                            color="heatwave_dwd_mean"
                           )
fig.update_geos(fitbounds="locations", visible=False)

fig.show()